<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Common Functions for `GiRaFFEfood` Initial Data for `GiRaFFE`

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Common_Functions.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Common_Functions.py)

**Notebook Status:** <font color='red'><b> In Progress </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module through the main initial data modules that depend on it.

## Introduction: 
We will need to "feed" our giraffe with initial data to evolve. There are several different choices of initial data we can use here; while each represents different physical systems, they all have some steps in common with each other. To avoid code duplication, we will first write several functions that we will use for all of them.

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#vectorpotential): Set the vector potential from input functions 
1. [Step 3](#velocity): Compute $v^i_{(n)}$ from $E^i$ and $B^i$
1. [Step 4](#setall): Generate specified initial data

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules, set the reference metric to Cartesian, and set commonly used NRPy+ parameters. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.

In [ ]:
# Step 0: Import the NRPy+ core modules and set the reference metric to Cartesian
import NRPy_param_funcs as par
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = __name__


<a id='vectorpotential'></a>

# Step 2: Set the vector potential from input functions \[Back to [top](#toc)\]
$$\label{vectorpotential}$$

First, we will write a function to generate the vector potential from input functions for each component. This function will also apply the correct coordinate staggering if the input is set as such. That is, in the staggered prescription, $A_x$ is sampled at $(i,j+1/2,k+1/2)$, $A_y$ at $(i+1/2,j,k+1/2)$, and $A_z$ at $(i+1/2,j+1/2,k)$.

We will first do this for initial data that are given with Cartesian vector components.

In [ ]:
# Generic function for all 1D tests: Compute Ax,Ay,Az
def Axyz_func_Cartesian(Ax_func,Ay_func,Az_func, stagger_enable, **params):
    x = rfm.xx_to_Cart[0]
    y = rfm.xx_to_Cart[1]
    z = rfm.xx_to_Cart[2]
    AD = ixp.zerorank1()
    # First Ax
    if stagger_enable:
        y += sp.Rational(1,2)*gri.dxx[1]
        z += sp.Rational(1,2)*gri.dxx[2]
    AD[0] = Ax_func(x,y,z, **params)
    # Then Ay
    if stagger_enable:
        x += sp.Rational(1,2)*gri.dxx[0]
        y -= sp.Rational(1,2)*gri.dxx[1]
        z += sp.Rational(1,2)*gri.dxx[2]
    AD[1] = Ay_func(x,y,z, **params)
    # Finally Az
    if stagger_enable:
        x += sp.Rational(1,2)*gri.dxx[0]
        y += sp.Rational(1,2)*gri.dxx[1]
        z -= sp.Rational(1,2)*gri.dxx[2]
    AD[2] = Az_func(x,y,z, **params)

    return AD

However, some initial data are given with spherical vector components. These will need to be handled slightly differently. We will need to perform a change of basis; fortunately, sympy provides us with powerful tools to do so. We will use these to first write a generic function to transform vectors to a Cartesian basis.

In [ ]:
# Generic function to convert contravariant vectors from a spherical to Cartesian basis.
def change_basis_spherical_to_Cartesian_D(somevector_sphD):
    # Use the Jacobian matrix to transform the vectors to Cartesian coordinates.
    drrefmetric__dx_0UDmatrix = sp.Matrix([[sp.diff(rfm.xxSph[0],rfm.xx[0]), sp.diff(rfm.xxSph[0],rfm.xx[1]), sp.diff(rfm.xxSph[0],rfm.xx[2])],
                                           [sp.diff(rfm.xxSph[1],rfm.xx[0]), sp.diff(rfm.xxSph[1],rfm.xx[1]), sp.diff(rfm.xxSph[1],rfm.xx[2])],
                                           [sp.diff(rfm.xxSph[2],rfm.xx[0]), sp.diff(rfm.xxSph[2],rfm.xx[1]), sp.diff(rfm.xxSph[2],rfm.xx[2])]])

    somevectorD = ixp.zerorank1()

    for i in range(3):
        for j in range(3):
            somevectorD[i] += drrefmetric__dx_0UDmatrix[(j,i)]*somevector_sphD[j]

    return somevectorD

# Generic function to convert covariant vectors from a spherical to Cartesian basis.
def change_basis_spherical_to_Cartesian_U(somevector_sphU):
    # Use the Jacobian matrix to transform the vectors to Cartesian coordinates.
    drrefmetric__dx_0UDmatrix = sp.Matrix([[sp.diff(rfm.xxSph[0],rfm.xx[0]), sp.diff(rfm.xxSph[0],rfm.xx[1]), sp.diff(rfm.xxSph[0],rfm.xx[2])],
                                           [sp.diff(rfm.xxSph[1],rfm.xx[0]), sp.diff(rfm.xxSph[1],rfm.xx[1]), sp.diff(rfm.xxSph[1],rfm.xx[2])],
                                           [sp.diff(rfm.xxSph[2],rfm.xx[0]), sp.diff(rfm.xxSph[2],rfm.xx[1]), sp.diff(rfm.xxSph[2],rfm.xx[2])]])
    dx__drrefmetric_0UDmatrix = drrefmetric__dx_0UDmatrix.inv()

    somevectorU = ixp.zerorank1()

    for i in range(3):
        for j in range(3):
            somevectorU[i] += dx__drrefmetric_0UDmatrix[(j,i)]*somevector_sphU[j]

    return somevectorU

In [ ]:
# Generic function for all 1D tests: Compute Ax,Ay,Az
def Axyz_func_spherical(Ar_func,At_func,Ap_func, stagger_enable, **params):
    r     = rfm.xxSph[0] + KerrSchild_radial_shift # We are setting the data up in Shifted Kerr-Schild coordinates
    theta = rfm.xxSph[1]
    phi   = rfm.xxSph[2]
    AsphD = ixp.zerorank1()
    # First Ax
    if stagger_enable:
        y += sp.Rational(1,2)*gri.dxx[1]
        z += sp.Rational(1,2)*gri.dxx[2]
    AsphD[0] = Ar_func(r,theta,phi, **params)
    # Then Ay
    if stagger_enable:
        x += sp.Rational(1,2)*gri.dxx[0]
        y -= sp.Rational(1,2)*gri.dxx[1]
        z += sp.Rational(1,2)*gri.dxx[2]
    AsphD[1] = At_func(r,theta,phi, **params)
    # Finally Az
    if stagger_enable:
        x += sp.Rational(1,2)*gri.dxx[0]
        y += sp.Rational(1,2)*gri.dxx[1]
        z -= sp.Rational(1,2)*gri.dxx[2]
    AsphD[2] = Ap_func(r,theta,phi, **params)

    # Use the Jacobian matrix to transform the vectors to Cartesian coordinates.
    AD = change_basis_spherical_to_Cartesian(AsphD)
    return AD

<a id='velocity'></a>

# Step 3: Compute $v^i_{(n)}$ from $E^i$ and $B^i$ \[Back to [top](#toc)\]
$$\label{velocity}$$

This function computes the Valenciea 3-velocity from input electric and magnetic fields. It can also take the three-metric $\gamma_{ij}$ as an optional input; if this is not set, the function defaults to flat spacetime.

In [ ]:
# Generic function for all 1D tests: Valencia 3-velocity from ED and BU
def compute_ValenciavU_from_ED_and_BU(ED, BU, gammaDD=None):
    # Now, we calculate v^i = ([ijk] E_j B_k) / B^2,
    # where [ijk] is the Levi-Civita symbol and B^2 = \gamma_{ij} B^i B^j$ is a trivial dot product in flat space.
    LeviCivitaSymbolDDD = ixp.LeviCivitaSymbol_dim3_rank3()

    B2 = sp.sympify(0)
    # In flat spacetime, use the Minkowski metric; otherwise, use the input metric.
    if gammaDD is None:
        gammaDD = ixp.zerorank2()
        for i in range(3):
            gammaDD[i][i] = sp.sympify(1)
    for i in range(3):
        for j in range(3):
            B2 += gammaDD[i][j] * BU[i] * BU[j]

    BD = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            BD[i] = gammaDD[i][j]*BU[j]
    ValenciavU = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                ValenciavU[i] += LeviCivitaSymbolDDD[i][j][k] * ED[j] * BD[k] / B2

    return ValenciavU

<a id='setall'></a>

# Step 4: Generate specified initial data \[Back to [top](#toc)\]
$$\label{setall}$$

This is the main function that users can call to generate the initial data by passing the name of the initial data as a string and specifying if they want to enable staggering.

In [ ]:
def GiRaFFEfood_NRPy_generate_initial_data(ID_type = "DegenAlfvenWave", stagger_enable = False,**params):
    global AD, ValenciavU
#     if ID_type == "DegenAlfvenWave":
#         mu_DAW = par.Cparameters("REAL",thismodule,["mu_DAW"], -0.5) # The wave speed
#         AD = Axyz_func(Ax_DAW, Ay_DAW, Az_DAW, stagger_enable,
#                        gammamu=sp.sympify(1)/sp.sqrt(sp.sympify(1)-mu_AW**2))
#         ValenciaVU = ValenciavU_DAW(mu_DAW=mu_DAW, gammamu=gammamu)
#     elif ID_type == "FastWave":
#         AD = Axyz_func(Ax_FW, Ay_FW, Az_FW, stagger_enable)
#         ValenciaVU = ValenciavU_FW()
    if ID_type == "SplitMonopole":
        import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Split_Monopole as gfsm
        AD = gfcf.Axyz_func_spherical(gfsm.Ar_SM,gfsm.Ath_SM,gfsm.Aph_SM,stagger_enable,**params)
        ValenciavU = gfsm.ValenciavU_func_SM(**params)